In [14]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [4]:
df = pd.read_csv("owid-co2-data.csv")

In [5]:
df



,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,co2_including_luc,co2_including_luc_growth_abs,co2_including_luc_growth_prct,co2_including_luc_per_capita,co2_including_luc_per_gdp,co2_including_luc_per_unit_energy,co2_per_capita,co2_per_gdp,co2_per_unit_energy,coal_co2,coal_co2_per_capita,consumption_co2,consumption_co2_per_capita,consumption_co2_per_gdp,cumulative_cement_co2,cumulative_co2,cumulative_co2_including_luc,cumulative_coal_co2,cumulative_flaring_co2,cumulative_gas_co2,cumulative_luc_co2,cumulative_oil_co2,cumulative_other_co2,energy_per_capita,energy_per_gdp,flaring_co2,flaring_co2_per_capita,gas_co2,gas_co2_per_capita,ghg_excluding_lucf_per_capita,ghg_per_capita,land_use_change_co2,land_use_change_co2_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,oil_co2,oil_co2_per_capita,other_co2_per_capita,other_industry_co2,primary_energy_consumption,share_global_cement_co2,share_global_co2,share_global_co2_including_luc,share_global_coal_co2,share_global_cumulative_cement_co2,share_global_cumulative_co2,share_global_cumulative_co2_including_luc,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_luc_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,share_global_flaring_co2,share_global_gas_co2,share_global_luc_co2,share_global_oil_co2,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1850,AFG,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.931,0.781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.121,NaN,NaN,NaN,NaN,0.121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1851,AFG,3767956.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.968,0.788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.119,NaN,NaN,NaN,NaN,0.118,NaN,NaN,0.165,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
2,Afghanistan,1852,AFG,3783940.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.968,0.784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.118,NaN,NaN,NaN,NaN,0.116,NaN,NaN,0.164,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
3,Afghanistan,1853,AFG,3800954.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.004,0.790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.118,NaN,NaN,NaN,NaN,0.115,NaN,NaN,0.164,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
4,Afghanistan,1854,AFG,3818038.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.004,0.787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.117,NaN,NaN,NaN,NaN,0.114,NaN,NaN,0.163,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50593,Zimbabwe,2017,ZWE,14751101.0,2.194784e+10,0.469,0.032,9.596,-0.937,-8.899,19.086,-1.817,-8.691,1.294,0.870,0.422,0.651,0.437,0.212,5.900,0.400,10.506,0.712,0.479,18.479,753.403,2974.117,598.525,0.0,0.0,2281.390,136.400,NaN,3068.011,2.062,0.0,0.0,0.0,0.0

In [6]:
df.columns

Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

In [7]:
df.shape

(50598, 79)

In [8]:
df[df['country'] == 'World']

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,co2_including_luc,co2_including_luc_growth_abs,co2_including_luc_growth_prct,co2_including_luc_per_capita,co2_including_luc_per_gdp,co2_including_luc_per_unit_energy,co2_per_capita,co2_per_gdp,co2_per_unit_energy,coal_co2,coal_co2_per_capita,consumption_co2,consumption_co2_per_capita,consumption_co2_per_gdp,cumulative_cement_co2,cumulative_co2,cumulative_co2_including_luc,cumulative_coal_co2,cumulative_flaring_co2,cumulative_gas_co2,cumulative_luc_co2,cumulative_oil_co2,cumulative_other_co2,energy_per_capita,energy_per_gdp,flaring_co2,flaring_co2_per_capita,gas_co2,gas_co2_per_capita,ghg_excluding_lucf_per_capita,ghg_per_capita,land_use_change_co2,land_use_change_co2_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,oil_co2,oil_co2_per_capita,other_co2_per_capita,other_industry_co2,primary_energy_consumption,share_global_cement_co2,share_global_co2,share_global_co2_including_luc,share_global_coal_co2,share_global_cumulative_cement_co2,share_global_cumulative_co2,share_global_cumulative_co2_including_luc,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_luc_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,share_global_flaring_co2,share_global_gas_co2,share_global_luc_co2,share_global_oil_co2,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
49810,World,1750,NaN,7.456641e+08,NaN,NaN,NaN,9.351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.013,NaN,NaN,9.351,0.013,NaN,NaN,NaN,NaN,9.351,NaN,9.351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,100.0,NaN,100.0,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49811,World,1751,NaN,NaN,NaN,NaN,NaN,9.351,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.351,NaN,NaN,NaN,NaN,NaN,18.701,NaN,18.701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,100.0,NaN,100.0,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49812,World,1752,NaN,NaN,NaN,NaN,NaN,9.354,0.004,0.039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.354,NaN,NaN,NaN,NaN,NaN,28.055,NaN,28.055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,100.0,NaN,100.0,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49813,World,1753,NaN,NaN,NaN,NaN,NaN,9.354,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.354,NaN,NaN,NaN,NaN,NaN,37.409,NaN,37.409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,100.0,NaN,100.0,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49814,World,1754,NaN,NaN,NaN,NaN,NaN,9.358,0.004,0.039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.358,NaN,NaN,NaN,NaN,NaN,46.767,NaN,46.767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,100.0,NaN,100.0,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50077,World,2017,NaN,7.599822e+09,1.104307e+14,1507.923,0.198,36096.738,572.547,1.612,40428.684,376.644,0.940,5.320,0.366,0.257,4.750,0.327,0.230,14506.974,1.909,36096.742,4.750,0.327,38570.070,1590633.000,2311825.75,745563.062,16778.807,223402.328,725755.125,557836

In [9]:
df.isna().sum()

country                            0
year                               0
iso_code                        8456
population                     10590
gdp                            36034
                               ...  
temperature_change_from_n2o    12978
total_ghg                      44449
total_ghg_excluding_lucf       44449
trade_co2                      46339
trade_co2_share                46340
Length: 79, dtype: int64

In [12]:
#Data pre-processing for doushboard purposes
#Fill NAs with 0s and create GDP per capital column
df = df.fillna(0)

#Eclude from the new column the 0 values 
df['gdp_per_capita'] = np.where(df['population'] != 0 , df['gdp']/ df['population'], 0 )
df['gdp_per_capita']

0           0.000000
1           0.000000
2           0.000000
3           0.000000
4           0.000000
            ...     
50593    1487.877848
50594    1509.106089
50595       0.000000
50596       0.000000
50597       0.000000
Name: gdp_per_capita, Length: 50598, dtype: float64

In [15]:
#Make Dataframe pipline interactive
idf = df.interactive()

### CO2 emissions over time by continent

In [51]:
#Define Panel widgets
year_slider = pn.widgets.IntSlider(name = 'Year slider',
                                   start=1750, 
                                   end = 2020, 
                                   step=5, 
                                   value=1850)


In [52]:
year_slider

BokehModel(combine_events=True, render_bundle={'docs_json': {'8edabfce-ce1d-4f44-b835-29c836946c8b': {'version…

In [23]:
#Radio buttions for CO2 measures
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['co2','co2_per_capita',],
    button_type = 'success'
)


In [25]:
continents = ['World', 'Asia', 'Oceana', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_pipeline = (idf[(idf.year <= year_slider) &
                    (idf.country.isin(continents))]
                .groupby(['country','year'])[yaxis_co2].mean()
                .to_frame()
                .reset_index()
                .sort_values(by='year')
                .reset_index(drop = True)
                )

co2_pipeline



BokehModel(combine_events=True, render_bundle={'docs_json': {'0ea65602-e787-4456-ac4d-08f383460ace': {'version…

In [ ]:
co2_plot = co2_pipeline.hvplot(x='year',
                               by = 'country',
                               y = yaxis_co2, 
                               line_width = 2,
                               title = 'CO2 emission by continent')
co2_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'49c4b53e-9d85-491a-8fc1-1e2c54e16fce': {'version…

### CO2 emissions over time by continent - Table

In [27]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator,
                              pagination = 'remote',
                              page_size = 10,
                              sizing_mode = 'stretch_width')
co2_table

BokehModel(combine_events=True, render_bundle={'docs_json': {'624e709e-6935-46a9-8be5-db29158276c9': {'version…

### CO2 vs GDP scatterplot

In [29]:
# Create another pipeline for scatterplot
co2_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) &
        (~(idf.country.isin(continents)))
        ]
                .groupby(['country','year','gdp_per_capita'])['co2'].mean()
                .to_frame()
                .reset_index()
                .sort_values(by='year')
                .reset_index(drop = True)
)

In [30]:
co2_vs_gdp_scatterplot_pipeline

BokehModel(combine_events=True, render_bundle={'docs_json': {'9f5b5be6-0f2e-411b-a3c2-ea9bbb524fea': {'version…

In [31]:
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x='gdp_per_capita',
                                                                y ='co2',
                                                                by = 'country',
                                                                size =80, 
                                                                kind = 'scatter',
                                                                alpha = 0.7,
                                                                legend = False,
                                                                height =500,
                                                                width =500)

co2_vs_gdp_scatterplot

BokehModel(combine_events=True, render_bundle={'docs_json': {'fcf45535-681f-436c-9f9f-c4772e5740ba': {'version…

### Bar chart with co2 sources by continent

In [32]:
yaxis_co2_sources = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['coal_co2', 'oil_co2', 'gas_co2'],
    button_type = 'success'
)

continents_excl_world = ['World', 'Asia', 'Oceana', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']
co2_source_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf.country.isin(continents_excl_world))
        ]
    .groupby(['country','year'])[yaxis_co2_sources].sum()
                .to_frame()
                .reset_index()
                .sort_values(by='year')
                .reset_index(drop = True)
)



In [33]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind = 'bar',
                                                     x = 'country',
                                                     y = yaxis_co2_sources,
                                                     title = 'CO2 source by continent')
co2_source_bar_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'21cf86e2-8596-4f3a-a666-e4e3d37d8521': {'version…

### Create Dashboard

In [50]:
#Layout using template from plan website
template = pn.template.FastListTemplate(
    title = 'World CO2 emission dashboard',
    sidebar = [pn.pane.Markdown("# CO2 Emissions and Climate Change"),
               pn.pane.Markdown('#### Carbon diaxode emissions are the primary delivery of the glibal climate change. '),
               pn.pane.PNG('climate_day.png', sizing_mode = 'scale_both'),
               pn.pane.Markdown("#### Settings"),
               year_slider],
    main =[pn.Row(pn.Column(yaxis_co2, co2_plot.panel(width=600, margin=(0,25))),
                  pn.Column(co2_table.panel(width =600, margin_left=(0,25)))),
           pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width =600), margin=(0,25)),
                  pn.Column(yaxis_co2_sources, co2_source_bar_plot.panel(width = 600)))],
           accent_base_color = "#88d8b0",
           header_background ="#88d8b0",       
                  )
    
template.show()
template.servable();

Launching server at http://localhost:51740
